In [1]:
import sys

# Increase recursion depth to handle the bitwise decomposition of large constraints.
# Since constraints are ~10^9, we expect around 30 recursive levels (log2 of 10^9).
sys.setrecursionlimit(5000)

MOD = 10**9 + 7
BASE_PRIMES = [2, 3, 5, 7, 11]

# --- Precomputation for Efficiency ---
# bit_weights[bit_index][variable_index] stores: BASE_PRIMES[variable_index]^(2^bit_index) % MOD
# This allows O(1) weight calculation for each bit position during recursion.
bit_weights = []
for k in range(32):
    row = [pow(p, 1 << k, MOD) for p in BASE_PRIMES]
    bit_weights.append(row)

def solve_weighted_polytope(initial_capacities):
    """
    Computes the weighted sum P = sum(Product(p_i ^ x_i)) for the polytope defined 
    by x_i + x_j <= capacity_m using Digit Dynamic Programming.
    """
    memo = {}

    def calculate_bit_contribution(remaining_capacities, bit_level):
        # Base Case: If any capacity becomes negative, this path is geometrically impossible.
        if any(cap < 0 for cap in remaining_capacities):
            return 0
        
        # Termination: If all capacities are 0, only x_i = 0 is possible for all remaining bits.
        if all(cap == 0 for cap in remaining_capacities):
            return 1
        
        # Memoization state: current constraints and the bit level we are processing.
        state = (remaining_capacities, bit_level)
        if state in memo:
            return memo[state]
        
        total_weighted_sum = 0
        
        # Try all 2^5 = 32 combinations for the bits of [x0, x1, x2, x3, x4] at the current bit_level.
        for mask in range(32):
            current_bits = [(mask >> i) & 1 for i in range(5)]
            
            is_valid_combination = True
            next_step_capacities = []
            constraint_idx = 0
            
            # Check the 10 pairwise constraints: x_i + x_j <= Capacity
            for i in range(5):
                for j in range(i + 1, 5):
                    sum_of_bits = current_bits[i] + current_bits[j]
                    
                    if sum_of_bits > remaining_capacities[constraint_idx]:
                        is_valid_combination = False
                        break
                    
                    # Calculate "Carry" for the next bit level:
                    # Next_Cap = floor((Current_Cap - bit_i - bit_j) / 2)
                    next_cap = (remaining_capacities[constraint_idx] - sum_of_bits) // 2
                    next_step_capacities.append(next_cap)
                    constraint_idx += 1
                
                if not is_valid_combination:
                    break
            
            if is_valid_combination:
                # Recursive call for the next bit significance level (k + 1)
                future_sum = calculate_bit_contribution(tuple(next_step_capacities), bit_level + 1)
                
                if future_sum > 0:
                    # Calculate the product of p_i^(bit_i * 2^bit_level)
                    current_weight = 1
                    for i in range(5):
                        if current_bits[i]:
                            current_weight = (current_weight * bit_weights[bit_level][i]) % MOD
                    
                    # Add (current_weight * future_sum) to the result
                    total_weighted_sum = (total_weighted_sum + (current_weight * future_sum)) % MOD
        
        memo[state] = total_weighted_sum
        return total_weighted_sum

    return calculate_bit_contribution(initial_capacities, 0)

def generate_pseudo_random_constraints(total_needed):
    """
    Generates a sequence of constraints using the recurrence: x_n = (7*x_{n-1} + x_{n-2}^2) % MOD
    """
    sequence = [0] * total_needed
    val_prev2, val_prev1 = 1, 7
    sequence[0], sequence[1] = val_prev2, val_prev1
    for i in range(2, total_needed):
        val_next = (7 * val_prev1 + val_prev2**2) % MOD
        sequence[i] = val_next
        val_prev2, val_prev1 = val_prev1, val_next
    return sequence

def main():
    num_polytopes = 100
    print("Generating constraint sequence...")
    all_constraints = generate_pseudo_random_constraints(10 * num_polytopes)
    
    final_accumulated_sum = 0
    print(f"Starting calculation for {num_polytopes} polytopes...")
    
    for i in range(num_polytopes):
        # Extract 10 constraints for the current polytope
        current_poly_bounds = tuple(all_constraints[10 * i : 10 * i + 10])
        
        polytope_result = solve_weighted_polytope(current_poly_bounds)
        final_accumulated_sum = (final_accumulated_sum + polytope_result) % MOD
        
        if (i + 1) % 10 == 0:
            print(f"Progress: {i + 1}/{num_polytopes} calculated. Sum so far: {final_accumulated_sum}")

    print("-" * 40)
    print(f"Final Combined Result: {final_accumulated_sum}")

if __name__ == "__main__":
    main()

Generating constraint sequence...
Starting calculation for 100 polytopes...
Progress: 10/100 calculated. Sum so far: 975879241
Progress: 20/100 calculated. Sum so far: 502237211
Progress: 30/100 calculated. Sum so far: 463213806
Progress: 40/100 calculated. Sum so far: 524761895
Progress: 50/100 calculated. Sum so far: 452269057
Progress: 60/100 calculated. Sum so far: 804280989
Progress: 70/100 calculated. Sum so far: 340644019
Progress: 80/100 calculated. Sum so far: 526425656
Progress: 90/100 calculated. Sum so far: 958751997
Progress: 100/100 calculated. Sum so far: 885362394
----------------------------------------
Final Combined Result: 885362394


# Weighted Polytope Integer Point Counting

This project implements an advanced algorithm to solve a weighted counting problem within a 5-dimensional polytope. It uses **Digit Dynamic Programming** (Bit-Decomposition) to handle large constraints and high-dimensional summations efficiently.

## 1. Problem Statement
The goal is to calculate a weighted sum over an integer lattice defined by 10 pairwise linear inequalities. Given variables $x_0, x_1, x_2, x_3, x_4$, we solve for:

$$P = \sum_{(x_0, \dots, x_4) \in \mathcal{S}} \left( \prod_{i=0}^{4} p_i^{x_i} \right) \pmod{10^9+7}$$

Where:
- $p_i$ are the first five prime numbers $\{2, 3, 5, 7, 11\}$.
- $\mathcal{S}$ is the set of non-negative integers satisfying $x_i + x_j \leq \text{Capacity}_{m}$.

---

## 2. Methodology: Digit-by-Digit DP
Since the capacities are as large as $10^9$, brute-force iteration is impossible. The algorithm decomposes each variable $x_i$ into its binary representation: $x_i = \sum \text{bit}_{i,k} \cdot 2^k$.

### Transition & Carry Logic
The algorithm processes bits from Least Significant (LSB) to Most Significant (MSB). For each bit level $k$:
1. We test all 32 combinations (using a `mask`) for the current bits of the 5 variables.
2. For each constraint $x_i + x_j \leq C$, we check if the selected bits exceed the current available capacity.
3. We calculate the "Carry" for the next bit level to ensure the inequality holds for higher powers of 2:
   $$\text{next\_cap} = \lfloor \frac{\text{remaining\_capacity} - (b_i + b_j)}{2} \rfloor$$



---

## 3. Code Structure & Key Variables

To ensure clarity during code review, the following variables represent the core mathematical logic:

* **`bit_weights[bit_level][i]`**: Precomputed values of $p_i^{2^{bit\_level}} \pmod{MOD}$. This allows the multiplicative weight to be calculated incrementally.
* **`solve_weighted_polytope`**: The entry point that takes the initial 10 constraints and starts the bitwise recursion.
* **`calculate_bit_contribution`**: The recursive engine (Digit DP). It explores valid bit combinations level by level.
* **`remaining_capacities`**: A tuple representing the state of the 10 constraints at the current bit level. This is the primary key for **Memoization**.
* **`mask`**: An integer (0-31) representing the chosen bits for $x_0 \dots x_4$ at the current level.
* **`future_sum`**: The result of the recursive call for the next bit significance level ($k+1$).
* **`current_weight`**: The weight contribution of the chosen bits at the current level: $\prod p_i^{b_{i,k} \cdot 2^k}$.

---

## 4. Efficiency
- **Memoization:** By storing the results of `(remaining_capacities, bit_level)`, we prune the state space significantly.
- **Complexity:** $O(2^5 \cdot \log(\text{Max\_Cap}) \cdot \text{Unique\_States})$.
- **Pruning:** Invalid paths (where bits exceed capacity) are terminated immediately, making the algorithm highly performant for constraints up to $10^9$.